## Drive Mount


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Working directory

In [ ]:
import os
os.chdir('./drive/MyDrive/KEMD/code')
os.getcwd()

## Train / Test file

In [ ]:
import pandas as pd

In [ ]:
#Train
train =pd.read_csv('../data/Train.csv',encoding='euc-kr')
#Test
test=pd.read_csv('../data/Test.csv',encoding='euc-kr')

## Package installation required

In [ ]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch     

In [ ]:
# Hugging Face API로 KoBERT 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

## Package Load

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
!pip3 install kobert-transformers

In [ ]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
# ●
#from transformers import BertTokenizer

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 설정
device = torch.device("cuda:0")

In [ ]:
#BERT, Vocabulary 불러오기
#bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

## Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [ ]:
# 레이블 인코딩
le = LabelEncoder()
le.fit(train.Emotion.values)
encoded_train_y = le.transform(train.Emotion.values)

# train_y에서 사용된 LabelEncoder 객체를 그대로 사용하여 test_y 인코딩
encoded_test_y = le.transform(test.Emotion.values)

train["Emotion"] = encoded_train_y

test["Emotion"] = encoded_test_y

## Bert Train / Test set



In [ ]:
#Bert Train set
train_data = []
for q, label in zip(train['Text'],train['Emotion']):
    data =[]
    data.append(q)
    data.append(str(label))

    train_data.append(data)

#Bert Test set
test_data=[]
for q, label in zip(test['Text'],test['Emotion']):
    data_1 =[]
    data_1.append(q)
    data_1.append(str(label))

    test_data.append(data_1)

## Bert Input Dataset class

In [ ]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab 

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [ ]:
# ★
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


## Bert hyperparameters

In [ ]:
# Bert hyperparameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 7
max_grad_norm = 1
log_interval = 200
learning_rate =  2e-05
dr_rate = 0.5

## Train Tokenization

In [ ]:
train_dataset = BERTDataset(train_data, 0, 1, tokenizer, vocab, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,num_workers=2)

## KoBERT

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=dr_rate,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

    def extract_features(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        output, _ = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), attention_mask=attention_mask.float().to(token_ids.device))
        return output

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=dr_rate).to(device)

## Optimizar & Schedule

In [ ]:
#Optimizar : AdamW
no_decay = ['bias', 'LayerNorm.weight']

optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

#Loss function
loss_fn = nn.CrossEntropyLoss()

#Scheduler
t_total = len(train_dataloader) * num_epochs

warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

## Metric : Accuracy

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

## Model Train

In [ ]:
#epoch 7
for e in range(num_epochs):
    train_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

## Model save

In [ ]:
model_save_dir = '../model/'
if not os.path.exists(model_save_dir):
    os.mkdir(model_save_dir)
torch.save(model, model_save_dir + 'kobert_train.pt') 
torch.save(model.state_dict(), model_save_dir + 'kobert_state_dict.pt')  # 모델 객체의 state_dict 저장

torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, model_save_dir + 'kobert_emotions_all.tar')

## Model Load

In [ ]:
# Train 안돌렸을 때 Optimizar & Schedule 다시 설정
# """## Optimizar & Schedule"""

# #Optimizar : AdamW
# no_decay = ['bias', 'LayerNorm.weight']

# optimizer_grouped_parameters = [
#     {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# ]

# optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# #Loss function
# loss_fn = nn.CrossEntropyLoss()

# #Scheduler
# t_total = len(train_dataloader) * num_epochs

# warmup_step = int(t_total * warmup_ratio)

# scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# #정확도 측정을 위한 함수 정의
# def calc_accuracy(X,Y):
#     max_vals, max_indices = torch.max(X, 1)
#     train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
#     return train_acc

In [ ]:
#Model load
model = torch.load('../model/kobert_train.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load('../model/kobert_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

checkpoint = torch.load('../model/kobert_emotions_all.tar')   # dict 불러오기
model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

## Model Test function

In [ ]:
def predict(test_data):
    another_test = BERTDataset(test_data, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=20)

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            test_eval.append(np.argmax(logits))
            for e in len(test_eval):
                submission['topic_idx'][e] = test_eval[e]

        print(test_eval)

## Train set Predict

In [ ]:
# #Train set 예측
# train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size)

# train_eval=[]
# train_prob=[]

# for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
#     token_ids = token_ids.long().to(device)
#     segment_ids = segment_ids.long().to(device)

#     valid_length= valid_length
#     label = label.long().to(device)

#     out = model(token_ids, valid_length, segment_ids)

#     for i in out:
#         logits=i
#         logits = logits.detach().cpu().numpy()
#         train_eval.append(np.argmax(logits))
#         probs = nn.functional.softmax(torch.tensor(logits).reshape(-1, 7), dim=1)
#         train_prob.extend(probs.detach().cpu().numpy())

# train_probabilty = np.stack(train_prob)

# np.savez('predict_train_prob.npz',predict_prob=train_probabilty)

## Model Test

In [ ]:
test_dataset = BERTDataset(test_data, 0, 1, tokenizer, vocab, max_len, True, False)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=2)


test_eval=[]
test_prob=[]

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)

    valid_length= valid_length
    label = label.long().to(device)

    out = model(token_ids, valid_length, segment_ids)

    for i in out:
        logits=i
        logits = logits.detach().cpu().numpy()
        test_eval.append(np.argmax(logits))
        probs = nn.functional.softmax(torch.tensor(logits).reshape(-1, 7), dim=1)
        test_prob.extend(probs.detach().cpu().numpy())

## Model evaluation

In [ ]:
# 예측값
predict= pd.DataFrame(test_eval)

y_pred = list(predict[0])

#predict.to_csv('kobert_predict.csv')

# 실제값 
y_true = list(test['Emotion'])

#Classification Report Test 성능 확인
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))

In [ ]:
"""### test 예측확률 저장"""

test_probabilty = np.stack(test_prob)

np.savez('KoBERT_'+str(dr_rate)[0]+str(dr_rate)[2]+'.npz',predict_prob=test_probabilty)

# data = np.load('predict_prob.npz')

## Embadding Visualization

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
eatures = []

with torch.no_grad():
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        feature = model.extract_features(token_ids, valid_length, segment_ids)
        features.append(feature.detach().cpu().numpy())

features = np.concatenate(features, axis=0)

# feature shape을 (6312, 64*768)으로 reshape
features_2d = features.reshape((features.shape[0], -1))

In [ ]:
# T-SNE 모델 생성 및 학습
tsne = TSNE(n_components=2, random_state=0)
features_tsne = tsne.fit_transform(features_2d)

In [ ]:
#Embadding Visulaztion
cmap = plt.cm.get_cmap('jet', 7)
emotion_list = ['angry', 'disqust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
plt.scatter(features_tsne[:, 0], features_tsne[:, 1], c=test['Emotion'], cmap=cmap, s=5)

# legend 달기
for i, label in enumerate(np.unique(test['Emotion'])):
    plt.scatter([], [], c=cmap(i), label=emotion_list[label], s=30)

# legend 위치 조정
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()
#plt.savefig('t-SNE_word_embadding visualization_test.jpg')

## Pretrained Model Embadding Visualization

In [ ]:
no_train_bert,no_train_vocab = get_pytorch_kobert_model()

no_train_model = BERTClassifier(no_train_bert,  dr_rate=0.5).to(device)

In [ ]:
eatures_1 = []

with torch.no_grad():
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        feature = no_train_model.extract_features(token_ids, valid_length, segment_ids)
        features_1.append(feature.detach().cpu().numpy())

features_1 = np.concatenate(features_1, axis=0)

# feature shape을 (6312, 64*768)으로 reshape
features_2d_1 = features_1.reshape((features_1.shape[0], -1))

In [ ]:
# T-SNE 모델 생성 및 학습
tsne = TSNE(n_components=2, random_state=0)
features_tsne_1 = tsne.fit_transform(features_2d_1)

In [ ]:
#Embadding Visulaztion
cmap = plt.cm.get_cmap('jet', 7)
emotion_list = ['angry', 'disqust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
plt.scatter(features_tsne_1[:, 0], features_tsne_1[:, 1], c=test['Emotion'], cmap=cmap, s=5)

# legend 달기
for i, label in enumerate(np.unique(test['Emotion'])):
    plt.scatter([], [], c=cmap(i), label=emotion_list[label], s=30)

# legend 위치 조정
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()
#plt.savefig('t-SNE_word_embadding visualization_pretrained_model_test.jpg')